In [0]:
%sql
-- Definição do catálogo e schema utilizados na camada Silver.

USE CATALOG mvp;
USE SCHEMA silver;

In [0]:
# Leitura da Delta Table criada na Camada Bronze, que contém os dados brutos padronizados

df_aero_bronze = spark.table("mvp.bronze.aeroportos")

In [0]:
# Para padronização e limpeza dos dados aplicamos o filtro que remove os aeroportos inativos sem relevância operacional
# e removemos as linhas sem UF informado, que impediriam integrações futuras e análises geográficas

from pyspark.sql.functions import col

df_aero_filtrado = (
    df_aero_bronze
    .filter(col("situacao") == "Cadastrado")
    .filter(col("uf").isNotNull())
)

In [0]:
# Definição das colunas relevantes para a Camada Silver

df_aero_silver = df_aero_filtrado.select(
    "icao_aerodromo",
    "codigo_ciad",
    "nome_aeroporto",
    "municipio",
    "uf",
    "latitude",
    "longitude"
)


In [0]:
# Criação da tabela Silver em formato delta para armazenar os dados

(
    df_aero_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .saveAsTable("mvp.silver.aeroportos")
)


In [0]:
%sql
-- Consulta para checagem da quantidade total de aeroportos válidos

SELECT COUNT(*) FROM mvp.silver.aeroportos;


COUNT(*)
467


In [0]:
%sql
-- Verificar se existem aeroportos com código ICAO duplicado (resultado esperado é nenhum retorno)

SELECT icao_aerodromo, COUNT(*)
FROM mvp.silver.aeroportos
GROUP BY icao_aerodromo
HAVING COUNT(*) > 1;


icao_aerodromo,COUNT(*)


In [0]:
%sql
-- Consulta para checagem da distribuição de aeroportos por UF

SELECT uf, COUNT(*) AS quantidade
FROM mvp.silver.aeroportos
GROUP BY uf
ORDER BY COUNT(*) DESC;


uf,quantidade
São Paulo,68
Minas Gerais,60
Rio Grande do Sul,44
Bahia,37
Paraná,33
Goiás,24
Pará,23
Mato Grosso,23
Amazonas,22
Santa Catarina,21


In [0]:
%sql
-- Consulta exploratória da Camada Silver Aeroportos

SELECT * 
FROM mvp.silver.aeroportos
LIMIT 50;

icao_aerodromo,codigo_ciad,nome_aeroporto,municipio,uf,latitude,longitude
SDZG,CE0008,Pedro Teixeira Castelo Regional Tauá,TAUÁ,Ceará,"05°56'00""S","040°17'51""W"
SDIG,SP0038,Ibitinga,IBITINGA,São Paulo,"21°44'50""S","048°51'21""W"
SBVT,ES0001,Eurico de Aguiar Salles,VITÓRIA,Espírito Santo,"20°15'29""S","040°17'11""W"
SNJK,BA0047,Jequié,JEQUIÉ,Bahia,"13°52'40""S","040°04'17""W"
SSKU,SC0016,Lauro Antônio da Costa,CURITIBANOS,Santa Catarina,"27°17'14""S","050°36'14""W"
SSEP,RS0031,São Sepé,SÃO SEPÉ,Rio Grande do Sul,"30°10'56""S","053°34'46""W"
SJRG,RS0013,Regional de Rio Grande,RIO GRANDE,Rio Grande do Sul,"32°05'01""S","052°10'04""W"
SSXX,SC0017,Municipal João Winckler,XANXERÊ,Santa Catarina,"26°52'30""S","052°22'22""W"
SBDO,MS0008,Dourados,DOURADOS,Mato Grosso do Sul,"22°12'02""S","054°55'32""W"
SNGD,PI0005,Guadalupe,GUADALUPE,Piauí,"06°46'59""S","043°34'51""W"
